# Collection of Data from ricksteves.com

## Import Necessary Libraries

In [1]:
import pandas as pd
import time
import urllib.request
from collections import Counter
from selenium.webdriver import Chrome
import pickle
import pymongo
import numpy as np
from scrape import collect_urls, collect_all_data, collect_city_data

### Instantiate MongoDB Databases

In [2]:
mc = pymongo.MongoClient()

In [3]:
city_db = mc['city_database']

In [4]:
city_collection = city_db['city_collection']
country_collection = city_db['country_collection']

### Web Scrape

In [5]:
browser = Chrome()
url = "https://www.ricksteves.com/"
browser.get(url)

In [6]:
browser.find_element_by_xpath('//*[@id="nav"]/ul/li[2]/a').click()

In [7]:
country_urls = collect_urls(browser, '/europe/')

In [8]:
country_urls

['https://www.ricksteves.com/europe/austria',
 'https://www.ricksteves.com/europe/belgium',
 'https://www.ricksteves.com/europe/bosnia-herzegovina',
 'https://www.ricksteves.com/europe/bulgaria',
 'https://www.ricksteves.com/europe/croatia',
 'https://www.ricksteves.com/europe/czech-republic',
 'https://www.ricksteves.com/europe/denmark',
 'https://www.ricksteves.com/europe/england',
 'https://www.ricksteves.com/europe/estonia',
 'https://www.ricksteves.com/europe/finland',
 'https://www.ricksteves.com/europe/france',
 'https://www.ricksteves.com/europe/germany',
 'https://www.ricksteves.com/europe/greece',
 'https://www.ricksteves.com/europe/hungary',
 'https://www.ricksteves.com/europe/iceland',
 'https://www.ricksteves.com/europe/ireland',
 'https://www.ricksteves.com/europe/italy',
 'https://www.ricksteves.com/europe/montenegro',
 'https://www.ricksteves.com/europe/netherlands',
 'https://www.ricksteves.com/europe/norway',
 'https://www.ricksteves.com/europe/poland',
 'https://www.

In [9]:
country_urls = country_urls[0:-2]

In [10]:
country_urls

['https://www.ricksteves.com/europe/austria',
 'https://www.ricksteves.com/europe/belgium',
 'https://www.ricksteves.com/europe/bosnia-herzegovina',
 'https://www.ricksteves.com/europe/bulgaria',
 'https://www.ricksteves.com/europe/croatia',
 'https://www.ricksteves.com/europe/czech-republic',
 'https://www.ricksteves.com/europe/denmark',
 'https://www.ricksteves.com/europe/england',
 'https://www.ricksteves.com/europe/estonia',
 'https://www.ricksteves.com/europe/finland',
 'https://www.ricksteves.com/europe/france',
 'https://www.ricksteves.com/europe/germany',
 'https://www.ricksteves.com/europe/greece',
 'https://www.ricksteves.com/europe/hungary',
 'https://www.ricksteves.com/europe/iceland',
 'https://www.ricksteves.com/europe/ireland',
 'https://www.ricksteves.com/europe/italy',
 'https://www.ricksteves.com/europe/montenegro',
 'https://www.ricksteves.com/europe/netherlands',
 'https://www.ricksteves.com/europe/norway',
 'https://www.ricksteves.com/europe/poland',
 'https://www.

Now that we have the url for all the countries, lets get the urls for each city. Let's start collecting some data. First go to the country's page.

In [11]:
for country in country_urls:
    collect_all_data(browser, country, country_collection, city_collection)

Inserted Austria into country collection
Inserted Danube Valley, Austria into city collection
Inserted Hallstatt, Austria into city collection
Inserted Salzburg, Austria into city collection
Inserted Tirol, Austria into city collection
Inserted Vienna, Austria into city collection
Completed scrapping Austria
Inserted Belgium into country collection
Inserted Antwerp, Belgium into city collection
Inserted Bruges, Belgium into city collection
Inserted Brussels, Belgium into city collection
Inserted Ghent, Belgium into city collection
Completed scrapping Belgium
Inserted Bosnia-Herzegovina into country collection
Inserted Mostar, Bosnia-Herzegovina into city collection
Inserted Sarajevo, Bosnia-Herzegovina into city collection
Completed scrapping Bosnia-Herzegovina
Inserted Bulgaria into country collection
Completed scrapping Bulgaria
Inserted Croatia into country collection
Inserted Dalmatian Coast, Croatia into city collection
Inserted Dubrovnik, Croatia into city collection
Inserted Hva

Inserted Pisa, Italy into city collection
Inserted Pompeii & Herculaneum, Italy into city collection
Inserted Ravenna, Italy into city collection
Inserted Rome, Italy into city collection
Inserted Sicily, Italy into city collection
Inserted Siena, Italy into city collection
Inserted Sorrento, Italy into city collection
Inserted Tuscan Hill Towns, Italy into city collection
Inserted Tuscany, Italy into city collection
Inserted Venice, Italy into city collection
Completed scrapping Italy
Inserted Montenegro into country collection
Completed scrapping Montenegro
Inserted Netherlands into country collection
Inserted Amsterdam, Netherlands into city collection
Inserted Delft, Netherlands into city collection
Inserted Edam, Netherlands into city collection
Inserted Haarlem, Netherlands into city collection
Inserted The Hague, Netherlands into city collection
Completed scrapping Netherlands
Inserted Norway into country collection
Inserted Bergen, Norway into city collection
Inserted Norwegian

Take the MongoDB collection and turn in into a list of dictionaries for the countries.

In [14]:
country_dicts = [x for x in country_collection.find()]
country_dicts

[{'_id': ObjectId('5d1106db2f3f9bdaae0a50ca'),
  'country': 'Austria',
  'country_summary': "Small, landlocked Austria offers alpine scenery, world-class museums, cobbled quaintness, and Wiener schnitzel. Unlike Germany, its industrious neighbor to the northwest, Austria is content to bask in its good living and elegant, opulent past as the former head of one of Europe's grandest empires. Austrians tend to be relaxed, gregarious people who love the outdoors as much as a good cup of coffee in a café."},
 {'_id': ObjectId('5d1106dd2f3f9bdaae0a50cb'),
  'country': 'Belgium',
  'country_summary': "Belgium falls through the cracks. Wedged between Germany, France, and the Netherlands, and famous for waffles, Smurfs, and a statue of a little boy peeing, it's no wonder it can get lost in the mix. But Belgium rewards with richer sights than you might expect — and fewer tourist crowds. You'll encounter some of Europe's finest cuisine, including the best beer, creamiest chocolates, and tastiest F

Because I scrapped the same countries a few times while I figured the bugs in my system, I want to make sure that I have each country only once. For example, I knew that there were only 31 countries but I have 55 entries in the `country_collection`. After cleaning, I have 31 as expected.

In [21]:
len(country_dicts)

55

In [25]:
country_df = pd.DataFrame(country_dicts)
country_df.drop('_id', axis=1, inplace=True)
country_df.drop_duplicates(inplace=True)
country_df.head()

,country,country_summary
0,Austria,"Small, landlocked Austria offers alpine scener..."
1,Belgium,Belgium falls through the cracks. Wedged betwe...
2,Bosnia-Herzegovina,Apart from the tragic way it separated from Yu...
3,Bulgaria,"Endearing, surprising Bulgaria is a rewarding ..."
4,Croatia,With thousands of miles of seafront and more t...


In [26]:
len(country_df)

31

Saving the dataframe to a pickle file for use in other notebooks.

In [29]:
#pickle.dump(country_df, open('countries.pkl', 'wb'))

Similar to the countries, convert `city_collection` to a list of dictionaries, drop the duplicate cities (from 232 to 213) and save to a pickle file for other notebooks.

In [15]:
city_dicts = [x for x in city_collection.find()]
city_dicts

[{'_id': ObjectId('5d110b165c4fbe2cc8624dfd'),
  'city': 'Danube Valley',
  'country': 'Austria',
  'city_summary': "The Danube is at its romantic best just west of Vienna. Mix a cruise with a bike ride through the Danube's Wachau Valley, lined with ruined castles, beautiful abbeys (including the glorious Melk Abbey), small towns, and vineyard upon vineyard. Much of the valley has a warm fairy-tale glow, but a trip here isn't complete without the chilling contrast of a visit to the Mauthausen concentration camp memorial."},
 {'_id': ObjectId('5d110b225c4fbe2cc8624dfe'),
  'city': 'Hallstatt',
  'country': 'Austria',
  'city_summary': 'Lovable Hallstatt is a tiny town bullied onto a ledge between a selfish mountain and a swan-ruled lake, with a waterfall ripping furiously through its middle. The big draws of Hallstatt are its village and its lakeside setting. Come here to relax, nibble, wander, and paddle. Beyond lies the Salzkammergut region, a gentle land of lakes, forested mountains,

In [16]:
len(city_dicts)

232

In [27]:
city_df = pd.DataFrame(city_dicts)
city_df.drop('_id', axis=1, inplace=True)
city_df.drop_duplicates(inplace=True)
city_df.head()

,city,city_summary,country
0,Danube Valley,The Danube is at its romantic best just west o...,Austria
1,Hallstatt,Lovable Hallstatt is a tiny town bullied onto ...,Austria
2,Salzburg,"Thanks to its charmingly preserved old town, s...",Austria
3,Tirol,Mountainous Tirol — in Austria's western panha...,Austria
4,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria


In [28]:
len(city_df)

213

In [30]:
#pickle.dump(city_df, open('cities.pkl', 'wb'))

In [6]:
browser.find_element_by_xpath('//*[@id="cms_section_3"]/a').click()

In [7]:
browser.find_element_by_xpath('//*[@id="body"]/div[2]/div[1]/div[3]/ul/li[1]/a').click()

In [8]:
urls = collect_urls(browser, '/watch-read-listen/')
urls

['https://www.ricksteves.com/watch-read-listen/video/tv-show',
 'https://www.ricksteves.com/watch-read-listen/video/travel-talks',
 'https://www.ricksteves.com/watch-read-listen/read/articles',
 'https://www.ricksteves.com/watch-read-listen/read/travel-news',
 'https://www.ricksteves.com/watch-read-listen/travel-blogs',
 'https://www.ricksteves.com/watch-read-listen/audio/radio',
 'https://www.ricksteves.com/watch-read-listen/audio/audio-europe',
 'https://www.ricksteves.com/watch-read-listen/audio/audio-tours',
 'https://www.ricksteves.com/watch-read-listen/read/articles#nav',
 'https://www.ricksteves.com/watch-read-listen/read/articles#',
 'https://www.ricksteves.com/watch-read-listen/read/articles/ten-great-seaside-bars',
 'https://www.ricksteves.com/watch-read-listen/read/articles/10-tips-for-traveling-as-a-political-act',
 'https://www.ricksteves.com/watch-read-listen/read/articles/affordable-europe',
 'https://www.ricksteves.com/watch-read-listen/read/tms/andorra-not-lost-in-the-

In [9]:
len(urls)

624

In [239]:
urls_update = urls[130:]

danube = urls_update[0]
urls_update = urls_update[1:]

hallstatt = urls_update[0]
urls_update = urls_update[1:]

salzburg = urls_update[0:3]
urls_update = urls_update[3:]

triol = urls_update[0:2]
urls_update = urls_update[2:]

vienna = urls_update[0:9]
urls_update = urls_update[9:]

antwerp = urls_update[0]
urls_update = urls_update[3:]

burges = urls_update[0]
urls_update = urls_update[1:]

brussels = urls_update[0:2]
urls_update = urls_update[4:]

ghent = urls_update[0]
urls_update = urls_update[1:]

mostar = urls_update[3]
urls_update = urls_update[7:]

dalmatian_coast = urls_update[0]
urls_update = urls_update[3:]

istria = urls_update[0]
urls_update = urls_update[1:]

dubrovnik = urls_update[0]
urls_update = urls_update[9:]

aarhus = urls_update[0]
urls_update = urls_update[1:]

aero = urls_update[0]
urls_update = urls_update[1:]

copenhagen = [urls_update[1]]
copenhagen.append(urls_update[3])
urls_update = urls_update[5:]

bath = urls_update[11:13]
urls_update = urls_update[13:]

glastonbury_wells = urls_update[0]
urls_update = urls_update[1:]

cotswolds = urls_update[0:2]
urls_update = urls_update[2:]

oxford = urls_update[0]
cambridge = urls_update[0]
urls_update = urls_update[1:]

stratford = urls_update[0]
urls_update = urls_update[1:]

blackpool = urls_update[0]
urls_update = urls_update[4:]

york = urls_update[0:2]
urls_update = urls_update[2:]

london = urls_update[0:10]
urls_update = urls_update[10:]

brighton = urls_update[0]
urls_update = urls_update[1:]

windsor = urls_update[0]
urls_update = urls_update[1:]

cornwall = urls_update[0]
urls_update = urls_update[1:]

portsmouth = urls_update[0]
urls_update = urls_update[1:]

tallin = urls_update[1]
urls_update = urls_update[2:]

In [240]:
helsinki = [urls_update[0]]
helsinki.append(urls_update[2])
urls_update = urls_update[3:]

In [241]:
alsace = urls_update[10]
urls_update = urls_update[11:]

In [242]:
strasbourg = urls_update[0]
urls_update = urls_update[1:]

In [243]:
burgundy = urls_update[0:3]
urls_update = urls_update[3:]

In [244]:
dordogne = urls_update[0:3]
urls_update = urls_update[3:]

In [245]:
french_alps = urls_update[0]
urls_update = urls_update[1:]

In [246]:
french_basque = urls_update[0:2]
urls_update = urls_update[2:]

In [247]:
french_riviera = urls_update[1:3]
urls_update = urls_update[3:]

In [248]:
nice = urls_update[0]
urls_update = urls_update[1:]

In [249]:
languedoc_roussillon = urls_update[0:2]
urls_update = urls_update[2:]

In [250]:
loire = urls_update[0:3]
urls_update = urls_update[3:]

In [251]:
lyon = urls_update[0]
urls_update = urls_update[1:]

In [252]:
brittany = urls_update[0]
urls_update = urls_update[1:]

In [214]:
d_day_beaches = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/mont-st-michel',
 'https://www.ricksteves.com/watch-read-listen/read/articles/utah-beach-normandy-d-day',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-art-of-parisian-living',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-artistic-draw-of-france',
 'https://www.ricksteves.com/watch-read-listen/read/articles/paris-montmartre-bohemian',
 'https://www.ricksteves.com/watch-read-listen/read/articles/circling-in-on-paris-arc-de-triomphe',
 'https://www.ricksteves.com/watch-read-listen/read/articles/eiffel-tower-visiting-tips',
 'https://www.ricksteves.com/watch-read-listen/read/articles/le-metro-de-paris',
 'https://www.ricksteves.com/watch-read-listen/read/articles/lessons-from-a-paris-winetasting',
 'https://www.ricksteves.com/watch-read-listen/read/articles/turkey-gave-finger',
 'https://www.ricksteves.com/watch-read-listen/read/articles/monets-gardens-in-giverny',
 'https://www.ricksteves.

In [215]:
mont_st_michel = urls_update[0]
urls_update = urls_update[2:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/the-art-of-parisian-living',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-artistic-draw-of-france',
 'https://www.ricksteves.com/watch-read-listen/read/articles/paris-montmartre-bohemian',
 'https://www.ricksteves.com/watch-read-listen/read/articles/circling-in-on-paris-arc-de-triomphe',
 'https://www.ricksteves.com/watch-read-listen/read/articles/eiffel-tower-visiting-tips',
 'https://www.ricksteves.com/watch-read-listen/read/articles/le-metro-de-paris',
 'https://www.ricksteves.com/watch-read-listen/read/articles/lessons-from-a-paris-winetasting',
 'https://www.ricksteves.com/watch-read-listen/read/articles/turkey-gave-finger',
 'https://www.ricksteves.com/watch-read-listen/read/articles/monets-gardens-in-giverny',
 'https://www.ricksteves.com/watch-read-listen/read/articles/paris-a-city-of-cozy-neighborhoods',
 'https://www.ricksteves.com/watch-read-listen/read/articles/kid-surrealism',
 'https://www.ri

In [216]:
paris = urls_update[0:22]
urls_update = urls_update[22:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/finding-nemo-in-nimes',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hill-towns-of-the-luberon',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-magic-of-aix-en-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/picnicking-in-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/seeing-provence-through-van-gogh-eyes',
 'https://www.ricksteves.com/watch-read-listen/read/articles/bridge-avignon',
 'https://www.ricksteves.com/watch-read-listen/read/articles/wines-of-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/tips-for-rr-in-a-rural-rental',
 'https://www.ricksteves.com/watch-read-listen/read/articles/under-the-roman-sun-in-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/verdun-france',
 'https://www.ricksteves.com/watch-read-listen/read/articles/reims-a-bubbly-day-trip-from-paris',
 'https://www.ricksteves.

In [217]:
luberon = urls_update[1]
urls_update = urls_update[2:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/the-magic-of-aix-en-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/picnicking-in-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/seeing-provence-through-van-gogh-eyes',
 'https://www.ricksteves.com/watch-read-listen/read/articles/bridge-avignon',
 'https://www.ricksteves.com/watch-read-listen/read/articles/wines-of-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/tips-for-rr-in-a-rural-rental',
 'https://www.ricksteves.com/watch-read-listen/read/articles/under-the-roman-sun-in-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/verdun-france',
 'https://www.ricksteves.com/watch-read-listen/read/articles/reims-a-bubbly-day-trip-from-paris',
 'https://www.ricksteves.com/watch-read-listen/read/articles/paris-itinerary',
 'https://www.ricksteves.com/watch-read-listen/read/articles/celebrating-with-the-christkind',
 'https://www.ricksteves.

In [218]:
aix_en_provence = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/picnicking-in-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/seeing-provence-through-van-gogh-eyes',
 'https://www.ricksteves.com/watch-read-listen/read/articles/bridge-avignon',
 'https://www.ricksteves.com/watch-read-listen/read/articles/wines-of-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/tips-for-rr-in-a-rural-rental',
 'https://www.ricksteves.com/watch-read-listen/read/articles/under-the-roman-sun-in-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/verdun-france',
 'https://www.ricksteves.com/watch-read-listen/read/articles/reims-a-bubbly-day-trip-from-paris',
 'https://www.ricksteves.com/watch-read-listen/read/articles/paris-itinerary',
 'https://www.ricksteves.com/watch-read-listen/read/articles/celebrating-with-the-christkind',
 'https://www.ricksteves.com/watch-read-listen/read/articles/german-cuisine-for-travelers',
 'https://www.ricksteves.

In [219]:
provence = urls_update[0:2]
provence.append(urls_update[3])
provence.append(urls_update[5])
urls_update = urls_update[2:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/bridge-avignon',
 'https://www.ricksteves.com/watch-read-listen/read/articles/wines-of-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/tips-for-rr-in-a-rural-rental',
 'https://www.ricksteves.com/watch-read-listen/read/articles/under-the-roman-sun-in-provence',
 'https://www.ricksteves.com/watch-read-listen/read/articles/verdun-france',
 'https://www.ricksteves.com/watch-read-listen/read/articles/reims-a-bubbly-day-trip-from-paris',
 'https://www.ricksteves.com/watch-read-listen/read/articles/paris-itinerary',
 'https://www.ricksteves.com/watch-read-listen/read/articles/celebrating-with-the-christkind',
 'https://www.ricksteves.com/watch-read-listen/read/articles/german-cuisine-for-travelers',
 'https://www.ricksteves.com/watch-read-listen/read/articles/whats-new-in-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/afterhours-at-a-german-stammtisch',
 'https://www.ricksteves.com/wa

In [220]:
avignon = urls_update[0]
urls_update = urls_update[4:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/verdun-france',
 'https://www.ricksteves.com/watch-read-listen/read/articles/reims-a-bubbly-day-trip-from-paris',
 'https://www.ricksteves.com/watch-read-listen/read/articles/paris-itinerary',
 'https://www.ricksteves.com/watch-read-listen/read/articles/celebrating-with-the-christkind',
 'https://www.ricksteves.com/watch-read-listen/read/articles/german-cuisine-for-travelers',
 'https://www.ricksteves.com/watch-read-listen/read/articles/whats-new-in-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/afterhours-at-a-german-stammtisch',
 'https://www.ricksteves.com/watch-read-listen/read/articles/oktoberfest-munich',
 'https://www.ricksteves.com/watch-read-listen/read/articles/castles-of-mad-king-ludwig-ii',
 'https://www.ricksteves.com/watch-read-listen/read/articles/characters-of-rothenburg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/fuessen-oberammergau-castles-bavaria-germany',
 'https

In [221]:
reims = urls_update[1]
urls_update = urls_update[6:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/afterhours-at-a-german-stammtisch',
 'https://www.ricksteves.com/watch-read-listen/read/articles/oktoberfest-munich',
 'https://www.ricksteves.com/watch-read-listen/read/articles/castles-of-mad-king-ludwig-ii',
 'https://www.ricksteves.com/watch-read-listen/read/articles/characters-of-rothenburg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/fuessen-oberammergau-castles-bavaria-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/dachau-and-a-gruess-gott-shop',
 'https://www.ricksteves.com/watch-read-listen/read/articles/germanys-fairytale-dream-town-rothenburg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hitlers-eagles-nest-at-berchtesgaden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munich-a-metropolis-with-smalltown-charm',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munich-madness-oktoberfest-and-beer-halls',
 'https://www.ricksteves.c

In [222]:
rothenburg = urls_update[3]
urls_update = urls_update[4:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/fuessen-oberammergau-castles-bavaria-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/dachau-and-a-gruess-gott-shop',
 'https://www.ricksteves.com/watch-read-listen/read/articles/germanys-fairytale-dream-town-rothenburg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hitlers-eagles-nest-at-berchtesgaden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munich-a-metropolis-with-smalltown-charm',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munich-madness-oktoberfest-and-beer-halls',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munichs-revival',
 'https://www.ricksteves.com/watch-read-listen/read/articles/nuernbergs-christmas-market',
 'https://www.ricksteves.com/watch-read-listen/read/articles/pondering-german-history-in-nuernberg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rising-to-the-top-in-germany',
 'https://www.ricksteve

In [223]:
bavarian_alps = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/dachau-and-a-gruess-gott-shop',
 'https://www.ricksteves.com/watch-read-listen/read/articles/germanys-fairytale-dream-town-rothenburg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hitlers-eagles-nest-at-berchtesgaden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munich-a-metropolis-with-smalltown-charm',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munich-madness-oktoberfest-and-beer-halls',
 'https://www.ricksteves.com/watch-read-listen/read/articles/munichs-revival',
 'https://www.ricksteves.com/watch-read-listen/read/articles/nuernbergs-christmas-market',
 'https://www.ricksteves.com/watch-read-listen/read/articles/pondering-german-history-in-nuernberg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rising-to-the-top-in-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/wunderbares-wuerzburg',
 'https://www.ricksteves.com/watch-read-listen

In [224]:
munich = urls_update[3:6]
urls_update = urls_update[6:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/nuernbergs-christmas-market',
 'https://www.ricksteves.com/watch-read-listen/read/articles/pondering-german-history-in-nuernberg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rising-to-the-top-in-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/wunderbares-wuerzburg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-day-trips',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-21st-century',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-looks-to-the-future-remembers-its-past',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-then-and-now',
 'https://www.ricksteves.com/watch-read-listen/read/articles/in-search-of-communist-berlin',
 'https://www.ricksteves.com/watch-read-listen/read/articles/seeing-history-unfold',
 'https://www.ricksteves.com/watch-read-listen/read/articles/unter-den-linden',
 'https://www.rick

In [225]:
nurnberg = urls_update[0:2]
urls_update = urls_update[3:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/wunderbares-wuerzburg',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-day-trips',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-21st-century',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-looks-to-the-future-remembers-its-past',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-then-and-now',
 'https://www.ricksteves.com/watch-read-listen/read/articles/in-search-of-communist-berlin',
 'https://www.ricksteves.com/watch-read-listen/read/articles/seeing-history-unfold',
 'https://www.ricksteves.com/watch-read-listen/read/articles/unter-den-linden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/black-forest-germany-discovering-romance',
 'https://www.ricksteves.com/watch-read-listen/read/articles/going-cuckoo-over-germanys-black-forest',
 'https://www.ricksteves.com/watch-read-listen/read/articles/naked-truth-about-europe',
 'https

In [226]:
wurzburg = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/berlin-day-trips',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-21st-century',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-looks-to-the-future-remembers-its-past',
 'https://www.ricksteves.com/watch-read-listen/read/articles/berlin-then-and-now',
 'https://www.ricksteves.com/watch-read-listen/read/articles/in-search-of-communist-berlin',
 'https://www.ricksteves.com/watch-read-listen/read/articles/seeing-history-unfold',
 'https://www.ricksteves.com/watch-read-listen/read/articles/unter-den-linden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/black-forest-germany-discovering-romance',
 'https://www.ricksteves.com/watch-read-listen/read/articles/going-cuckoo-over-germanys-black-forest',
 'https://www.ricksteves.com/watch-read-listen/read/articles/naked-truth-about-europe',
 'https://www.ricksteves.com/watch-read-listen/read/articles/dresdens-inspiring-rebirth',
 '

In [227]:
berlin = urls_update[0:6]
urls_update = urls_update[7:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/black-forest-germany-discovering-romance',
 'https://www.ricksteves.com/watch-read-listen/read/articles/going-cuckoo-over-germanys-black-forest',
 'https://www.ricksteves.com/watch-read-listen/read/articles/naked-truth-about-europe',
 'https://www.ricksteves.com/watch-read-listen/read/articles/dresdens-inspiring-rebirth',
 'https://www.ricksteves.com/watch-read-listen/read/articles/erfurt-simple-unspoiled-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/great-sights-and-new-insights-in-dresden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/leipzig-iron-curtain',
 'https://www.ricksteves.com/watch-read-listen/read/articles/visiting-lutherland-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/exploring-frankfurt-germanys-hub',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hamburg-germany-undiscovered',
 'https://www.ricksteves.com/watch-read-listen/read/a

In [228]:
black_forest = urls_update[0:2]
urls_update = urls_update[2:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/naked-truth-about-europe',
 'https://www.ricksteves.com/watch-read-listen/read/articles/dresdens-inspiring-rebirth',
 'https://www.ricksteves.com/watch-read-listen/read/articles/erfurt-simple-unspoiled-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/great-sights-and-new-insights-in-dresden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/leipzig-iron-curtain',
 'https://www.ricksteves.com/watch-read-listen/read/articles/visiting-lutherland-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/exploring-frankfurt-germanys-hub',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hamburg-germany-undiscovered',
 'https://www.ricksteves.com/watch-read-listen/read/articles/cologne-perfume-chocolate-god',
 'https://www.ricksteves.com/watch-read-listen/read/articles/historic-trier',
 'https://www.ricksteves.com/watch-read-listen/read/articles/three-rhineland-castles',
 '

In [229]:
baden_baden = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/dresdens-inspiring-rebirth',
 'https://www.ricksteves.com/watch-read-listen/read/articles/erfurt-simple-unspoiled-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/great-sights-and-new-insights-in-dresden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/leipzig-iron-curtain',
 'https://www.ricksteves.com/watch-read-listen/read/articles/visiting-lutherland-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/exploring-frankfurt-germanys-hub',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hamburg-germany-undiscovered',
 'https://www.ricksteves.com/watch-read-listen/read/articles/cologne-perfume-chocolate-god',
 'https://www.ricksteves.com/watch-read-listen/read/articles/historic-trier',
 'https://www.ricksteves.com/watch-read-listen/read/articles/three-rhineland-castles',
 'https://www.ricksteves.com/watch-read-listen/read/articles/memories-of-war-in-bacharach'

In [230]:
dresden = [urls_update[0]]
dresden.append(urls_update[2])
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/erfurt-simple-unspoiled-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/great-sights-and-new-insights-in-dresden',
 'https://www.ricksteves.com/watch-read-listen/read/articles/leipzig-iron-curtain',
 'https://www.ricksteves.com/watch-read-listen/read/articles/visiting-lutherland-germany',
 'https://www.ricksteves.com/watch-read-listen/read/articles/exploring-frankfurt-germanys-hub',
 'https://www.ricksteves.com/watch-read-listen/read/articles/hamburg-germany-undiscovered',
 'https://www.ricksteves.com/watch-read-listen/read/articles/cologne-perfume-chocolate-god',
 'https://www.ricksteves.com/watch-read-listen/read/articles/historic-trier',
 'https://www.ricksteves.com/watch-read-listen/read/articles/three-rhineland-castles',
 'https://www.ricksteves.com/watch-read-listen/read/articles/memories-of-war-in-bacharach',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-river-raging-with-his

In [231]:
frankfurt = urls_update[4]
urls_update = urls_update[5:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/hamburg-germany-undiscovered',
 'https://www.ricksteves.com/watch-read-listen/read/articles/cologne-perfume-chocolate-god',
 'https://www.ricksteves.com/watch-read-listen/read/articles/historic-trier',
 'https://www.ricksteves.com/watch-read-listen/read/articles/three-rhineland-castles',
 'https://www.ricksteves.com/watch-read-listen/read/articles/memories-of-war-in-bacharach',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-river-raging-with-history',
 'https://www.ricksteves.com/watch-read-listen/read/articles/slowing-down-in-mosel-valley',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-castles-a-turret-studded-meander-along-the-rhine',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-bright-side-of-greece-lower-prices-fewer-tourists',
 'https://www.ricksteves.com/watch-read-listen/read/articles/easter-in-greece-lamb-and-gusto',
 'https://www.ricksteves.com/watch-read-l

In [232]:
hamburg = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/cologne-perfume-chocolate-god',
 'https://www.ricksteves.com/watch-read-listen/read/articles/historic-trier',
 'https://www.ricksteves.com/watch-read-listen/read/articles/three-rhineland-castles',
 'https://www.ricksteves.com/watch-read-listen/read/articles/memories-of-war-in-bacharach',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-river-raging-with-history',
 'https://www.ricksteves.com/watch-read-listen/read/articles/slowing-down-in-mosel-valley',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-castles-a-turret-studded-meander-along-the-rhine',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-bright-side-of-greece-lower-prices-fewer-tourists',
 'https://www.ricksteves.com/watch-read-listen/read/articles/easter-in-greece-lamb-and-gusto',
 'https://www.ricksteves.com/watch-read-listen/read/articles/a-feast-of-greek-pleasures',
 'https://www.ricksteves.com/watch-read-lis

In [233]:
cologne = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/historic-trier',
 'https://www.ricksteves.com/watch-read-listen/read/articles/three-rhineland-castles',
 'https://www.ricksteves.com/watch-read-listen/read/articles/memories-of-war-in-bacharach',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-river-raging-with-history',
 'https://www.ricksteves.com/watch-read-listen/read/articles/slowing-down-in-mosel-valley',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-castles-a-turret-studded-meander-along-the-rhine',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-bright-side-of-greece-lower-prices-fewer-tourists',
 'https://www.ricksteves.com/watch-read-listen/read/articles/easter-in-greece-lamb-and-gusto',
 'https://www.ricksteves.com/watch-read-listen/read/articles/a-feast-of-greek-pleasures',
 'https://www.ricksteves.com/watch-read-listen/read/articles/athens-a-new-look-for-an-old-city',
 'https://www.ricksteves.com/watch-read

In [234]:
mosel_valley = [urls_update[0]]
mosel_valley.append(urls_update[4])
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/three-rhineland-castles',
 'https://www.ricksteves.com/watch-read-listen/read/articles/memories-of-war-in-bacharach',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-river-raging-with-history',
 'https://www.ricksteves.com/watch-read-listen/read/articles/slowing-down-in-mosel-valley',
 'https://www.ricksteves.com/watch-read-listen/read/articles/rhine-castles-a-turret-studded-meander-along-the-rhine',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-bright-side-of-greece-lower-prices-fewer-tourists',
 'https://www.ricksteves.com/watch-read-listen/read/articles/easter-in-greece-lamb-and-gusto',
 'https://www.ricksteves.com/watch-read-listen/read/articles/a-feast-of-greek-pleasures',
 'https://www.ricksteves.com/watch-read-listen/read/articles/athens-a-new-look-for-an-old-city',
 'https://www.ricksteves.com/watch-read-listen/read/articles/history-comes-alive-in-modern-athens',
 'https://www.ric

In [235]:
rhine_valley = [urls_update[0]]
rhine_valley.append(urls_update[2])
rhine_valley.append(urls_update[4])
urls_update = urls_update[7:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/a-feast-of-greek-pleasures',
 'https://www.ricksteves.com/watch-read-listen/read/articles/athens-a-new-look-for-an-old-city',
 'https://www.ricksteves.com/watch-read-listen/read/articles/history-comes-alive-in-modern-athens',
 'https://www.ricksteves.com/watch-read-listen/read/articles/exploring-greeces-ancient-past-delphi-epidavros-and-mycenae',
 'https://www.ricksteves.com/watch-read-listen/read/articles/samaria-gorge',
 'https://www.ricksteves.com/watch-read-listen/read/articles/greek-island-hopping-basics',
 'https://www.ricksteves.com/watch-read-listen/read/articles/cock-crow-on-hydra',
 'https://www.ricksteves.com/watch-read-listen/read/articles/mykonos-a-greek-island-treasure',
 'https://www.ricksteves.com/watch-read-listen/read/articles/santorini-greece-island',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-peloponnese-greeces-unspoiled-escape',
 'https://www.ricksteves.com/watch-read-listen/read/ar

In [236]:
athens = urls_update[1:3]
urls_update = urls_update[3:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/exploring-greeces-ancient-past-delphi-epidavros-and-mycenae',
 'https://www.ricksteves.com/watch-read-listen/read/articles/samaria-gorge',
 'https://www.ricksteves.com/watch-read-listen/read/articles/greek-island-hopping-basics',
 'https://www.ricksteves.com/watch-read-listen/read/articles/cock-crow-on-hydra',
 'https://www.ricksteves.com/watch-read-listen/read/articles/mykonos-a-greek-island-treasure',
 'https://www.ricksteves.com/watch-read-listen/read/articles/santorini-greece-island',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-peloponnese-greeces-unspoiled-escape',
 'https://www.ricksteves.com/watch-read-listen/read/articles/where-the-ancient-olympics-meet-the-wild-west',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-city-of-paradox',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-life-among-the-ruins',
 'https://www.ricksteves.com/watch-read-listen/read

In [237]:
delphi = urls_update[0]
urls_update = urls_update[1:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/samaria-gorge',
 'https://www.ricksteves.com/watch-read-listen/read/articles/greek-island-hopping-basics',
 'https://www.ricksteves.com/watch-read-listen/read/articles/cock-crow-on-hydra',
 'https://www.ricksteves.com/watch-read-listen/read/articles/mykonos-a-greek-island-treasure',
 'https://www.ricksteves.com/watch-read-listen/read/articles/santorini-greece-island',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-peloponnese-greeces-unspoiled-escape',
 'https://www.ricksteves.com/watch-read-listen/read/articles/where-the-ancient-olympics-meet-the-wild-west',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-city-of-paradox',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-life-among-the-ruins',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-remembers-the-cold-war',
 'https://www.ricksteves.com/watch-read-listen/read/articles/enchanting-eger',


In [238]:
greek_islands = urls_update[0:2]
urls_update = urls_update[2:]
urls_update

['https://www.ricksteves.com/watch-read-listen/read/articles/mykonos-a-greek-island-treasure',
 'https://www.ricksteves.com/watch-read-listen/read/articles/santorini-greece-island',
 'https://www.ricksteves.com/watch-read-listen/read/articles/the-peloponnese-greeces-unspoiled-escape',
 'https://www.ricksteves.com/watch-read-listen/read/articles/where-the-ancient-olympics-meet-the-wild-west',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-city-of-paradox',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-life-among-the-ruins',
 'https://www.ricksteves.com/watch-read-listen/read/articles/budapest-remembers-the-cold-war',
 'https://www.ricksteves.com/watch-read-listen/read/articles/enchanting-eger',
 'https://www.ricksteves.com/watch-read-listen/read/articles/taking-the-plunge-in-budapest',
 'https://www.ricksteves.com/watch-read-listen/read/articles/10-ways-iceland-can-kill-you',
 'https://www.ricksteves.com/watch-read-listen/read/articles/icel

In [ ]:
Inserted Greek Islands, Greece into city collection
Inserted Hydra, Greece into city collection
Inserted Kardamyli, Greece into city collection
Inserted Mani Peninsula, Greece into city collection
Inserted Monemvasia, Greece into city collection
Inserted Mykonos, Greece into city collection
Inserted Nafplio, Greece into city collection
Inserted Olympia, Greece into city collection
Inserted Peloponnese, Greece into city collection
Inserted Santorini, Greece into city collection

Inserted Budapest, Hungary into city collection
Inserted Eger, Hungary into city collection
Inserted Aran Islands, Ireland into city collection
Inserted Belfast, Ireland into city collection
Inserted Connemara & County Mayo, Ireland into city collection
Inserted County Clare & the Burren, Ireland into city collection
Inserted County Donegal, Ireland into city collection
Inserted Derry, Ireland into city collection
Inserted Dingle Peninsula, Ireland into city collection
Inserted Dublin, Ireland into city collection
Inserted Galway, Ireland into city collection
Inserted Kenmare & the Ring of Kerry, Ireland into city collection
Inserted Kilkenny & the Rock of Cashel, Ireland into city collection
Inserted Kinsale & Cobh, Ireland into city collection
Inserted Northern Ireland, Ireland into city collection
Inserted Portrush & the Antrim Coast, Ireland into city collection
Inserted Waterford & County Wexford, Ireland into city collection
Inserted Amalfi Coast, Italy into city collection
Inserted Assisi, Italy into city collection
Inserted Capri, Italy into city collection
Inserted Cinque Terre, Italy into city collection
Inserted Civita di Bagnoregio, Italy into city collection
Inserted Dolomites, Italy into city collection
Inserted Florence, Italy into city collection
Inserted Italian Lakes, Italy into city collection
Inserted Italian Riviera, Italy into city collection
Inserted Lucca, Italy into city collection
Inserted Milan, Italy into city collection
Inserted Naples, Italy into city collection
Inserted Orvieto, Italy into city collection
Inserted Pisa, Italy into city collection
Inserted Pompeii & Herculaneum, Italy into city collection
Inserted Ravenna, Italy into city collection
Inserted Rome, Italy into city collection
Inserted Sicily, Italy into city collection
Inserted Siena, Italy into city collection
Inserted Sorrento, Italy into city collection
Inserted Tuscan Hill Towns, Italy into city collection
Inserted Tuscany, Italy into city collection
Inserted Venice, Italy into city collection
Inserted Montenegro into country collection
Inserted Netherlands into country collection
Inserted Amsterdam, Netherlands into city collection
Inserted Delft, Netherlands into city collection
Inserted Edam, Netherlands into city collection
Inserted Haarlem, Netherlands into city collection
Inserted The Hague, Netherlands into city collection
Inserted Norway into country collection
Inserted Bergen, Norway into city collection
Inserted Norwegian Fjords, Norway into city collection
Inserted Oslo, Norway into city collection
Inserted Poland into country collection
Inserted Auschwitz-Birkenau, Poland into city collection
Inserted Gdańsk, Poland into city collection
Inserted Kraków, Poland into city collection
Inserted Warsaw, Poland into city collection
Inserted Portugal into country collection
Inserted Algarve, Portugal into city collection
Inserted Coimbra, Portugal into city collection
Inserted Douro Valley, Portugal into city collection
Inserted Lisbon, Portugal into city collection
Inserted Nazaré, Portugal into city collection
Inserted Porto, Portugal into city collection
Inserted Sintra, Portugal into city collection
Inserted Évora, Portugal into city collection
Inserted Óbidos, Portugal into city collection
Inserted Russia into country collection
Inserted St. Petersburg, Russia into city collection
Inserted Scotland into country collection
Inserted Edinburgh, Scotland into city collection
Inserted Glasgow, Scotland into city collection
Inserted Isle of Skye, Scotland into city collection
Inserted Oban, Mull & Iona, Scotland into city collection
Inserted Scottish Highlands, Scotland into city collection
Inserted St. Andrews, Scotland into city collection
Inserted Bratislava, Slovakia into city collection
Inserted Julian Alps, Slovenia into city collection
Inserted Lake Bled, Slovenia into city collection
Inserted Ljubljana, Slovenia into city collection
Inserted Andalucía, Spain into city collection
Inserted Barcelona, Spain into city collection
Inserted Camino de Santiago, Spain into city collection
Inserted Costa del Sol, Spain into city collection
Inserted Córdoba, Spain into city collection
Inserted Granada, Spain into city collection
Inserted Madrid, Spain into city collection
Inserted Pamplona, Spain into city collection
Inserted Salamanca, Spain into city collection
Inserted San Sebastián, Spain into city collection
Inserted Santiago de Compostela, Spain into city collection
Inserted Sevilla, Spain into city collection
Inserted Spanish Basque Country, Spain into city collection
Inserted Toledo, Spain into city collection
Inserted White Hill Towns, Spain into city collection
Inserted Kalmar, Sweden into city collection
Inserted Stockholm, Sweden into city collection
Inserted Switzerland into country collection
Inserted Appenzell, Switzerland into city collection
Inserted Bern, Switzerland into city collection
Inserted Berner Oberland, Switzerland into city collection
Inserted Gimmelwald, Switzerland into city collection
Inserted Lake Geneva & French Switzerland, Switzerland into city collection
Inserted Luzern, Switzerland into city collection
Inserted Zermatt, Switzerland into city collection
Inserted Zürich, Switzerland into city collection
Inserted Ephesus, Turkey into city collection
Inserted Istanbul, Turkey into city collection
Inserted North Wales, Wales into city collection
Inserted South Wales, Wales into city collection